In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from skimage.feature import graycomatrix, graycoprops
from skimage.exposure import rescale_intensity
from skimage.filters import threshold_otsu
from tqdm import tqdm

EXCEL_PATH = "D:/MRI/Clinical_and_Other_Features.xlsx"         
patient_folder_root = "/Duke-Breast-Cancer-MRI/"       

df_excel = pd.read_excel(EXCEL_PATH)

df_excel["Date of Birth (Years)"] = df_excel["Date of Birth (Days)"].abs() // 365

all_patient_features = []

In [ ]:

def resample_to_match(source, target_shape):
    zoom_factors = [t / s for s, t in zip(source.shape, target_shape)]
    return zoom(source, zoom_factors, order=1)

def normalize_volume(img, mask):
    vals = img[mask]
    mean = np.mean(vals)
    std = np.std(vals)
    return (img - mean) / (std + 1e-6)

def compute_symmetry_index(image, mask):
    mid = image.shape[0] // 2
    left = image[:mid, :, :]
    right = image[mid:, :, :]
    right_flipped = np.flip(right, axis=0)

    shape = tuple(map(min, left.shape, right_flipped.shape))
    left = left[:shape[0], :shape[1], :shape[2]]
    right_flipped = right_flipped[:shape[0], :shape[1], :shape[2]]

    mask_left = mask[:shape[0], :shape[1], :shape[2]]
    mask_right = mask[mid:mid+shape[0], :shape[1], :shape[2]]
    mask_right = np.flip(mask_right, axis=0)
    valid_mask = mask_left & mask_right
    l_vals = left[valid_mask]
    r_vals = right_flipped[valid_mask]
    if len(l_vals) == 0 or len(r_vals) == 0:
        return np.nan  

    numerator = np.sum(np.abs(l_vals - r_vals))
    denominator = np.sum(np.abs(l_vals) + np.abs(r_vals)) + 1e-6

    return 1 - (numerator / denominator)


def compute_texture_features(region):
    region_scaled = rescale_intensity(region, out_range=(0, 255)).astype(np.uint8)
    slice_2d = region_scaled[:, :, region.shape[2] // 2]
    glcm = graycomatrix(slice_2d, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    entropy = -np.sum(glcm * np.log2(glcm + 1e-6))
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    return entropy, contrast, homogeneity

def compute_texture_features_3slice(region):
    region_scaled = rescale_intensity(region, out_range=(0, 255)).astype(np.uint8)
    center_slice_idx = region.shape[2] // 2
    slice_indices = [center_slice_idx - 1, center_slice_idx, center_slice_idx + 1]

    entropies = []
    contrasts = []
    homogeneities = []

    for idx in slice_indices:
        if 0 <= idx < region.shape[2]:
            slice_2d = region_scaled[:, :, idx]
            glcm = graycomatrix(slice_2d, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
            entropies.append(-np.sum(glcm * np.log2(glcm + 1e-6)))
            contrasts.append(graycoprops(glcm, 'contrast')[0, 0])
            homogeneities.append(graycoprops(glcm, 'homogeneity')[0, 0])

    return np.mean(entropies), np.mean(contrasts), np.mean(homogeneities)


def extract_features_with_breast_mask(patient_id, patient_dir):
    pre = nib.load(os.path.join(patient_dir, "pre.nii.gz")).get_fdata()
    post = nib.load(os.path.join(patient_dir, "post.nii.gz")).get_fdata()
    t1 = nib.load(os.path.join(patient_dir, "t1.nii.gz")).get_fdata()
    mask = nib.load(os.path.join(patient_dir, "breast_mask.nii.gz")).get_fdata()
    mask = mask > 0

    target_shape = post.shape
    pre = resample_to_match(pre, target_shape)
    t1 = resample_to_match(t1, target_shape)
    mask = resample_to_match(mask.astype(np.float32), target_shape) > 0.5
    pre = normalize_volume(pre, mask)
    post = normalize_volume(post, mask)
    t1 = normalize_volume(t1, mask)

    pre[pre == 0] = 1e-3
    enhancement_map = (post - pre) / pre
    enhancement_map[~mask] = 0

    max_clip = np.percentile(enhancement_map[mask], 99.9)
    enhancement_map = np.clip(enhancement_map, 0, max_clip)

    threshold = np.percentile(enhancement_map[mask], 98)
    tumor_mask = (enhancement_map > threshold) & mask
    tumor_region = enhancement_map[tumor_mask]

    mean_enh = np.mean(tumor_region)
    max_enh = np.max(tumor_region)
    std_enh = np.std(tumor_region)
    median_enh = np.median(tumor_region)
    skew_enh = pd.Series(tumor_region).skew()
    kurt_enh = pd.Series(tumor_region).kurt()
    tumor_volume = np.sum(tumor_mask)

    pre_safe = np.copy(pre)
    denominator = t1 - pre_safe
    valid_ser_mask = (denominator > 1)
    denominator[~valid_ser_mask] = 1
    ser_map = (post - pre_safe) / denominator
    ser_map[~mask] = 0
    mean_ser = np.mean(ser_map[tumor_mask & valid_ser_mask])

    breast_volume = np.sum(mask)

    t1_vals = t1[mask]
    thresh = threshold_otsu(t1_vals)
    dense_tissue = t1_vals > thresh
    density = np.sum(dense_tissue) / len(t1_vals)
    symmetry_index  = compute_symmetry_index(t1, mask)
    t1_tumor_overlap = t1[tumor_mask]
    mean_t1_tumor = np.mean(t1_tumor_overlap)
    entropy, contrast, homogeneity = compute_texture_features(post * mask)

    return {
        "Patient ID": patient_id,
        "MeanEnhancement": mean_enh,
        "MaxEnhancement": max_enh,
        "EnhancementSTD": std_enh,
        "EnhancementMedian": median_enh,
        "EnhancementSkew": skew_enh,
        "EnhancementKurtosis": kurt_enh,
        "TumorVolume": tumor_volume,
        "MeanSER": mean_ser,
        "BreastVolume": breast_volume,
        "BreastDensity": density,
        "SymmetryIndex": symmetry_index,
        "T1TumorMean": mean_t1_tumor,
        "GLCM_Entropy": entropy,
        "GLCM_Contrast": contrast,
        "GLCM_Homogeneity": homogeneity
    }
    

In [ ]:
# Updated batch processing loop with conditional skipping based on missing critical labels

excel_path = "D:/MRI/Clinical_and_Other_Features.xlsx"
patient_folder_root = "E:/2.Experiments_MRI/MRI_Data/Duke-Breast-Cancer-MRI/"

df_excel = pd.read_excel(excel_path)
df_excel["Date of Birth (Years)"] = df_excel["Date of Birth (Days)"].abs() // 365

required_labels = [
    "ER", "PR", "HER2", "Mol Subtype",
    "Staging(Tumor Size)# [T]",
    "Staging(Nodes)#(Nx replaced by -1)[N]",
    "Staging(Metastasis)#(Mx -replaced by -1)[M]",
    "Tumor Grade T", "Tumor Grade N", "Tumor Grade M",
    "Nottingham grade", "Histologic type",
    "Multicentric/Multifocal", "Skin/Nipple Invovlement",
    "Contralateral Breast Involvement", "Pec/Chest Involvement",
    "Recurrence event(s)"
]

all_features = []

for idx, row in df_excel.iterrows():
    patient_id = str(row["Patient ID"])
    recurrence = row.get("Recurrence event(s)", None)

    # Check skip condition
    if recurrence == 0:
        if any(pd.isna(row.get(label)) for label in required_labels):
            print(f"Skipping {patient_id}: Missing labels with recurrence = 0")
            continue

    patient_path = os.path.join(patient_folder_root, patient_id)

    try:
        features = extract_features_with_breast_mask(patient_id, patient_path)
        # Merge clinical info
        features.update({label: row.get(label, np.nan) for label in required_labels})
        features["Age"] = row.get("Date of Birth (Years)", np.nan)
        all_features.append(features)
        print(features)
    except Exception as e:
        print(f"Error processing {patient_id}: {e}")

# Save to CSV
df_final = pd.DataFrame(all_features)
df_final.to_csv("breast_mri_relapse_features.csv", index=False)


In [ ]:

annotation_file_path = ""
annotations = pd.read_excel(annotation_file_path)

# Function to extract tumor features from all tumor slices
def extract_tumor_features_from_box_all_slices(post_volume, box):
    sr, er = box['Start Row'], box['End Row']
    sc, ec = box['Start Column'], box['End Column']
    ss, es = box['Start Slice'], box['End Slice']

    tumor_crop_3d = post_volume[sr:er, sc:ec, ss:es]
    tumor_crop_flat = tumor_crop_3d[tumor_crop_3d > 0]

    if tumor_crop_flat.size == 0:
        return {
            "TumorMeanIntensity": np.nan,
            "TumorMaxIntensity": np.nan,
            "TumorStdIntensity": np.nan,
            "TumorVolumeVoxels": 0,
            "TumorEntropy": np.nan,
            "TumorContrast": np.nan,
            "TumorHomogeneity": np.nan
        }, None

    mean_intensity = np.mean(tumor_crop_flat)
    max_intensity = np.max(tumor_crop_flat)
    std_intensity = np.std(tumor_crop_flat)
    volume_voxels = tumor_crop_flat.size

    # Texture features across all slices
    entropies, contrasts, homogeneities = [], [], []
    for idx in range(ss, es):
        slice_2d = post_volume[sr:er, sc:ec, idx]
        if slice_2d.size == 0 or np.all(slice_2d == 0):
            continue
        slice_scaled = rescale_intensity(slice_2d, out_range=(0, 255)).astype(np.uint8)
        glcm = graycomatrix(slice_scaled, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        entropies.append(-np.sum(glcm * np.log2(glcm + 1e-6)))
        contrasts.append(graycoprops(glcm, 'contrast')[0, 0])
        homogeneities.append(graycoprops(glcm, 'homogeneity')[0, 0])

    entropy = np.mean(entropies) if entropies else np.nan
    contrast = np.mean(contrasts) if contrasts else np.nan
    homogeneity = np.mean(homogeneities) if homogeneities else np.nan

    features = {
        "TumorMeanIntensity": mean_intensity,
        "TumorMaxIntensity": max_intensity,
        "TumorStdIntensity": std_intensity,
        "TumorVolumeVoxels": volume_voxels,
        "TumorEntropy": entropy,
        "TumorContrast": contrast,
        "TumorHomogeneity": homogeneity
    }

    return features, tumor_crop_3d

def process_and_save_all_tumors(patient_root_dir, output_dir, post_filename="post.nii.gz"):
    os.makedirs(output_dir, exist_ok=True)
    saved_features = []

    for _, row in tqdm(annotations.iterrows(), total=len(annotations)):
        patient_id = row["Patient ID"]
        patient_path = os.path.join(patient_root_dir, patient_id)
        post_path = os.path.join(patient_path, post_filename)

        if not os.path.exists(post_path):
            print(f"Missing post.nii.gz for {patient_id}")
            continue

        try:
            post_volume = nib.load(post_path).get_fdata()
            tumor_features, tumor_crop = extract_tumor_features_from_box_all_slices(post_volume, row)

            if tumor_crop is not None:
                # Save cropped tumor volume as NIfTI
                tumor_nifti = nib.Nifti1Image(tumor_crop, affine=np.eye(4))
                save_path = os.path.join(output_dir, f"{patient_id}_tumor.nii.gz")
                nib.save(tumor_nifti, save_path)

            tumor_features["Patient ID"] = patient_id
            saved_features.append(tumor_features)
        except Exception as e:
            print(f"Error processing {patient_id}: {e}")

    return pd.DataFrame(saved_features)


In [ ]:
# Parameters for execution (update these with your actual local paths when running)
patient_folder_root = "/Duke-Breast-Cancer-MRI/"
output_tumor_dir = "Cropped_Tumor_Slices"

# Run the processing and save the tumor-specific features
tumor_features_df = process_and_save_all_tumors(patient_folder_root, output_tumor_dir)

# Save to CSV
tumor_features_csv_path = os.path.join(output_tumor_dir, "tumor_specific_features.csv")
tumor_features_df.to_csv(tumor_features_csv_path, index=False)

tumor_features_df.head()